In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import confusion_matrix
import os
from tensorflow.keras import utils
from imgaug import augmenters as iaa
# 進度條模組
from tqdm import tqdm
%matplotlib inline

# Load Data

In [ ]:
# 5種類
class_names = ['normal'  , 'error_iron' , 'error_ear' , 'noiron' , 'noearing']
# 7種類
# class_names = ['normal' , 'error_ear' , 'error_iron' , 'error_solder' , 'noearing' , 'noiron' , 'NG']

class_names_label = {class_name:i for i,class_name in enumerate(class_names)}
print(class_names_label)

num_class = len(class_names)

IMAGE_SIZE = (240 , 130)

In [ ]:
def load_data():
#     5 label
    datasets = ['E:/mask_data/New_mask_data_5/train' , 'E:/mask_data/New_mask_data_5/validation' , 'E:/mask_data/New_mask_data_5/test']
#     8 label
#     datasets = ['E:/mask_data/New_mask_data/train' , 'E:/mask_data/New_mask_data/validation' , 'E:/mask_data/New_mask_data/test']
    output = []
    
    for dataset in datasets:
        images = []
        labels = []
        
        print('Loading{}'.format(dataset))
        if dataset == datasets[0]:
            for folder in os.listdir(dataset):
                label = class_names_label[folder]

                for file in (os.listdir(os.path.join(dataset , folder))):


                    img_path = os.path.join(os.path.join(dataset , folder), file)

                    image = cv2.imread(img_path)
                    image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
                    image = cv2.resize(image , IMAGE_SIZE)
                    images.append(image)
                    labels.append(label)

                    flipper = iaa.Fliplr(1.0) #水平翻轉機率==1.0
                    image_f = flipper.augment_image(image) 
                    images.append(image_f)
                    labels.append(label)

                    vflipper = iaa.Flipud(1.0) #垂直翻轉機率40%
                    image_v = vflipper.augment_image(image) 
                    images.append(image_v)
                    labels.append(label)

            images = np.array(images , dtype = 'float32')
            labels = np.array(labels , dtype = 'int32')
            labels = utils.to_categorical(labels , num_class , dtype = 'int32')
            output.append((images,labels))
        else:
            for folder in os.listdir(dataset):
                label = class_names_label[folder]

                for file in (os.listdir(os.path.join(dataset , folder))):


                    img_path = os.path.join(os.path.join(dataset , folder), file)

                    image = cv2.imread(img_path)
                    image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
                    image = cv2.resize(image , IMAGE_SIZE)
                    images.append(image)
                    labels.append(label)

            images = np.array(images , dtype = 'float32')
            labels = np.array(labels , dtype = 'int32')
            labels = utils.to_categorical(labels , num_class , dtype = 'int32')
            output.append((images,labels))
        
    return output 

In [ ]:
from tensorflow.keras import utils
(train_images, train_labels), (val_images, val_labels), (test_images, test_labels) = load_data()

In [ ]:
from sklearn.utils import shuffle           

train_images, train_labels = shuffle(train_images, train_labels, random_state=25)
val_images, val_labels = shuffle(val_images, val_labels, random_state=25)
test_images, test_labels = shuffle(test_images, test_labels, random_state = 25)

In [ ]:
# check how many training and testing example do we have
# check the size of image

n_train = train_labels.shape[0]
n_val = val_labels.shape[0]
n_test = test_labels.shape[0]

print('Number of training example : {}'.format(n_train))
print('Number of training example : {}'.format(n_val))
print('Number of testing example : {}'.format(n_test))
print('Each image is of size : {}'.format(IMAGE_SIZE))

In [ ]:
train_images = train_images / 255.0
val_images = val_images / 255.0
test_images = test_images / 255.0

In [ ]:
# 灰階專用
train_images = tf.expand_dims(train_images, axis=-1)
val_images = tf.expand_dims(val_images, axis=-1)
test_images = tf.expand_dims(test_images, axis=-1)

# Data Pipline

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))

BATCH_SIZE = 8

train_dataset = train_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)

print(train_dataset)
print(val_dataset)

# Image Data Generator

In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# batch_size = 20

# datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip = True,rescale = 1/255)
# test_datagen = ImageDataGenerator(rescale = 1/255)


# Train_batches = datagen.flow(train_images , train_labels , batch_size = batch_size)

# Val_batches = test_datagen.flow(val_images , val_labels , batch_size = batch_size)

# Test_batches = test_datagen.flow(test_images , test_labels , batch_size = batch_size)

In [ ]:
# print(Train_batches.n)

In [ ]:
# if Train_batches.n % Train_batches.batch_size ==0:
#     step_size_train=Train_batches.n//Train_batches.batch_size
# else:
#     step_size_train=Train_batches.n//Train_batches.batch_size + 1
# print(step_size_train)

In [ ]:
# if Val_batches.n % Val_batches.batch_size ==0:
#     step_size_val=Val_batches.n//Val_batches.batch_size
# else:
#     step_size_val=Val_batches.n//Val_batches.batch_size + 1
# print(step_size_val)

# Create Model

In [ ]:
from tensorflow.keras.layers import Dense, Activation, Flatten ,Dropout ,BatchNormalization
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

input_layer = Input(shape=(130 ,240 ,1))

#Model
model = tf.keras.applications.EfficientNetB7(include_top = False , weights = None ,input_tensor = input_layer, classifier_activation = 'softmax')
model.summary()

In [ ]:
last_layer = model.output
last_layer = tf.keras.layers.GlobalAveragePooling2D()(last_layer)
last_layer = tf.keras.layers.Dropout(0.2)(last_layer)
out = Dense(num_class, activation='softmax', name='softmax')(last_layer)
model = Model(input_layer, out,name = 'EfficientNetB7-5label')

model.summary()

# Data Training

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import time

start = time.perf_counter()

Adam = tf.keras.optimizers.Adam(learning_rate=0.00025,name='Adam')

# earlystop = EarlyStopping(monitor='val_loss', mode='min', 
#                           verbose=1, patience=5, min_delta=0.0001)

filepath = 'Mask-5label-EfficientNetB7.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=1, verbose=1, mode='min', min_lr=0.0001)

# callback_list = [checkpoint , reduce_lr , earlystop]
callback_list = [checkpoint , reduce_lr]
# callback_list = [reduce_lr , earlystop]


model.compile(loss='categorical_crossentropy',optimizer=Adam,metrics=['accuracy'])

history = model.fit(train_dataset, epochs=15, verbose=1, validation_data = val_dataset,callbacks = callback_list)

# history = model.fit(Train_batches , steps_per_epoch=step_size_train , 
#                               validation_data = Val_batches , validation_steps = step_size_val ,
#                              epochs=80 , callbacks = callback_list)

In [ ]:
import math

end = time.perf_counter()

print('Time：'+str(math.floor(end - start)))

In [ ]:
def plot_accuracy_loss(history):
    """
        Plot the accuracy and the loss during the training of the nn.
    """
    fig = plt.figure(figsize=(10,5))

    # Plot accuracy
    plt.subplot(221)
    plt.plot(history.history['accuracy'],'b-', label = "acc")
    plt.plot(history.history['val_accuracy'], 'r-', label = "val_acc")
    plt.title("train_acc vs val_acc")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()

    # Plot loss function
    plt.subplot(222)
    plt.plot(history.history['loss'],'b-', label = "loss")
    plt.plot(history.history['val_loss'], 'r-', label = "val_loss")
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()
    plt.show()

In [ ]:
plot_accuracy_loss(history)

# Data Prediction

In [ ]:
model = tf.keras.models.load_model(filepath)

predictions = model.predict(test_images)
# predictions = model.predict_generator(Test_batches)     

pred_labels = np.argmax(predictions, axis = 1) 
pred_labels = np.array(pred_labels,dtype = 'int32')
pred_labels

In [ ]:
labels = np.argmax(test_labels, axis = 1) 
labels

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          cmap=plt.cm.Oranges):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    title='Mask-5label-EfficientNetB7 confusion matrix'
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
#     plt.savefig('E://Mask_confusion//Mask-EfficientNetB0.jpg')
    plt.show()


confusion_mtx = confusion_matrix(labels, pred_labels)
plot_confusion_matrix(confusion_mtx, classes = range(num_class))

In [ ]:
results = model.evaluate(test_images, test_labels, batch_size=128)
print("test loss, test acc:", results)

In [ ]:
count = 0
fail = 0

for i in range(208):
    count+=1
    if pred_labels[i]!=labels[i]:
        fail+=1
print(count)
print(fail)

In [ ]:
c = 0
for i in pred_labels:
    c+=1
print(c)
64+28